In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1) # 变量的初始值为截断正态分布
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, w):
    """
    tf.nn.conv2d功能：给定4维的input和filter，计算出一个2维的卷积结果
    前几个参数分别是input, filter, strides, padding, use_cudnn_on_gpu, ...
    input   的格式要求为一个张量，[batch, in_height, in_width, in_channels],批次数，图像高度，图像宽度，通道数
    filter  的格式为[filter_height, filter_width, in_channels, out_channels]，滤波器高度，宽度，输入通道数，输出通道数
    strides 一个长为4的list. 表示每次卷积以后在input中滑动的距离
    padding 有SAME和VALID两种选项，表示是否要保留不完全卷积的部分。如果是SAME，则保留
    use_cudnn_on_gpu 是否使用cudnn加速。默认是True
    """
    #stides: 1, stride x, stride y, 1
    #padding: SAME, VALID
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    """
    tf.nn.max_pool 进行最大值池化操作,而avg_pool 则进行平均值池化操作
    几个参数分别是：value, ksize, strides, padding,
    value:  一个4D张量，格式为[batch, height, width, channels]，与conv2d中input格式一样
    ksize:  长为4的list,表示池化窗口的尺寸
    strides: 窗口的滑动值，与conv2d中的一样
    padding: 与conv2d中用法一样。
    """
    #ksize: 1, kernel width x, kernel width y, 1
    #strides=1, stride x, stride y, 1
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [9]:
# 声明一个占位符，None表示输入图片的数量不定，28*28图片分辨率 
xs = tf.placeholder(tf.float32, [None, 28*28])
# 类别是0-9总共10个类别，对应输出分类结果  
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
# x_image又把xs reshape成了28*28*1的形状，因为是灰色图片，所以通道是1.作为训练时的input，-1代表图片数量不定  
x_image = tf.reshape(xs, [-1, 28, 28, 1])


#kernel=5x5, input_size=32, output_size=64
w_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
conv2 = tf.nn.relu(conv2d(pool1, w_conv2) + b_conv2) #output size 14*14*64
pool2 = max_pool_2x2(conv2)  #output size 7*7*64

#function1 layer
pool3 = tf.reshape(pool2, [-1, 7*7*64]) #[n_samples, 7, 7, 64] >> [n_sample, 7*7*64]
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
out3 = tf.nn.relu(tf.matmul(pool3, w_fc1) + b_fc1)

#function2 layer
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(out3, w_fc2) + b_fc2)


cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
  if i%100 ==0:
    print ('step:',i,',','accuracy:',compute_accuracy(mnist.test.images, mnist.test.labels))

step: 0 , accuracy: 0.1312
step: 100 , accuracy: 0.8956
step: 200 , accuracy: 0.9274
step: 300 , accuracy: 0.9483
step: 400 , accuracy: 0.9524
step: 500 , accuracy: 0.9619
step: 600 , accuracy: 0.9568
step: 700 , accuracy: 0.97
step: 800 , accuracy: 0.9719
step: 900 , accuracy: 0.9743


In [10]:
batch_xs.shape

(100, 784)

In [11]:
batch_ys.shape

(100, 10)